In [ ]:
# Import libraries
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# Get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

# Import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding="ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'}
)

df_ratings = pd.read_csv(
    ratings_filename,
    encoding="ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'}
)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# Filter users and books based on the number of ratings
user_counts = df_ratings['user'].value_counts()
users_to_keep = user_counts[user_counts >= 200].index

book_counts = df_ratings['isbn'].value_counts()
books_to_keep = book_counts[book_counts >= 100].index

filtered_ratings = df_ratings[df_ratings['user'].isin(users_to_keep) & df_ratings['isbn'].isin(books_to_keep)]

In [ ]:
# Function to return recommended books
def get_recommends(book=""):
    if book not in df_books['title'].values:
        return f"Book '{book}' not found."

    # Get the ISBN of the book
    isbn = df_books[df_books['title'] == book]['isbn'].values[0]

    # Find the index of the book in the ratings matrix
    book_index = ratings_matrix.columns.get_loc(isbn)

    # Get distances and indices of the nearest neighbors
    distances, indices = knn.kneighbors(ratings_sparse[:, book_index], n_neighbors=6)

    # Prepare the list of recommended books
    recommended_books = []
    for i in range(1, len(distances.flatten())):  # Skip the first one as it's the book itself
        book_isbn = ratings_matrix.columns[indices.flatten()[i]]
        book_title = df_books[df_books['isbn'] == book_isbn]['title'].values[0]
        recommended_books.append([book_title, distances.flatten()[i]])

    return [book, recommended_books]


In [ ]:
# Test the recommendation system
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
    test_pass = True
    recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
    if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
        test_pass = False
    recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
    recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
    for i in range(2):
        if recommends[1][i][0] not in recommended_books:
            test_pass = False
        if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
            test_pass = False
    if test_pass:
        print("You passed the challenge! 🎉🎉🎉🎉🎉")
    else:
        print("You haven't passed yet. Keep trying!")

test_book_recommendation()


In [ ]:
# Create a pivot table for ratings
ratings_matrix = filtered_ratings.pivot(index='user', columns='isbn', values='rating').fillna(0)
ratings_sparse = csr_matrix(ratings_matrix)

In [ ]:
# Train the Nearest Neighbors model
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6)
knn.fit(ratings_sparse)
